# Zeckendorf's Theorem and Property Based Testing

Reading [Donald Knuth's Concrete Mathematics](https://www.amazon.com/Concrete-Mathematics-Foundation-Computer-Science/dp/0201558025), I was delighted to learn about the wonderfully named [Zeckendorf's Theorem](https://en.wikipedia.org/wiki/Zeckendorf%27s_theorem). From Wikipedia, Zeckendorf's Theorem states

>Every positive integer can be represented uniquely as the sum of one or more Fibonacci numbers in such a way that the sum does not include any two consecutive Fibonacci numbers.

## An Implementation

We'll look at a simple implementation that returns the Zeckendorf representation of a number

In [1]:
from typing import List

def fibonacci_numbers_up_to(num: int) -> List[int]:
    """Returns a sorted list of fibonacci numbers up to a given number"""
    fibonacci_nums = [0, 1]
    next_fibonacci = fibonacci_nums[-2] + fibonacci_nums[-1]
    while next_fibonacci < num:
        fibonacci_nums.append(next_fibonacci)
        next_fibonacci = fibonacci_nums[-2] + fibonacci_nums[-1]

    return fibonacci_nums

def first_element_lte(list_: List[float], num: int) -> int:
    """Returns the first element in a list less than or equal to given number

    Raises ValueError if no elements in the list are less than or equal
    """
    for element in list_:
        if element <= num:
            return element
    else:
        raise ValueError()

def zeckendorf_representation(num: int) -> List[int]:
    """Returns a sorted list of fibonacci numbers that sum to a given number"""
    reversed_fibonaccis = reversed(fibonacci_numbers_up_to(num))

    # use a Greedy algorithm 
    zeckendorf_numbers = []
    while num > 0:
        next_zeckendorf = first_element_lte(reversed_fibonaccis, num)
        zeckendorf_numbers.append(next_zeckendorf)
        num -= next_zeckendorf

    return list(reversed(zeckendorf_numbers))


assert sum(zeckendorf_representation(100)) == 100
assert zeckendorf_representation(64) == [1, 8, 55]

## Testing

In [3]:
from hypothesis import given, settings, strategies

@given(strategies.integers(min_value=0))
def test_zeckendorf_representation(num: int):
    """Tests two properties of Zeckendorf Representation

    They sum up to the given number
    The sum does not include any consecutive Fibonacci numbers
    """
    zeckendorf_nums = zeckendorf_representation(num)
    assert sum(zeckendorf_nums) == num

    fibonacci_nums = fibonacci_numbers_up_to(num+1)
    # get the indices 
    indices = [
        fibonacci_nums.index(zeckendorf_num)
        for zeckendorf_num in zeckendorf_nums
    ]
    # check that no two indices are adjacent
    for index, next_index in zip(indices, indices[1:]):
        assert index + 1 < next_index


test_zeckendorf_representation()

Falsifying example: test_zeckendorf_representation(
    num=2,
)


AssertionError: 

### Debugging

Hmm... Hypothesis has found a bug for n=2. Let's see what we've got

In [4]:
zeckendorf_representation(2)

[1, 1]

Well that doesn't look right at all. 2 itself is a Fibonacci number and its Zeckendorf representation is simply `[2]`. Careful inspection shows that the bug is on the very first line of `zeckendorf_representation`.